# Structured Outputs with Amazon Bedrock in AG2


This notebook demonstrates how to use **structured outputs** with Amazon Bedrock in AG2. Structured outputs allow you to define a specific JSON schema that the model must follow, ensuring consistent and parseable responses.

## What are Structured Outputs?

Structured outputs enable you to:
- **Define a schema**: Specify exactly what format you want the model's response in
- **Get consistent results**: The model will always return data matching your schema
- **Parse easily**: Responses are guaranteed to be valid JSON matching your structure
- **Validate automatically**: Use Pydantic models to validate and type-check responses

## How Bedrock Implements Structured Outputs

Bedrock uses **Tool Use** (Function Calling) to implement structured outputs. When you provide a `response_format`, AG2:

1. Creates a special tool with your schema as the input schema
2. Forces the model to call this tool using `toolChoice`
3. Extracts the structured data from the tool call
4. Validates it against your Pydantic model or dict schema

This approach is based on the [AWS Bedrock Converse API](https://aws.amazon.com/blogs/machine-learning/structured-data-response-with-amazon-bedrock-prompt-engineering-and-tool-use/).

## Requirements

- Python >= 3.10
- AG2 installed: `pip install ag2`
- `boto3` package: `pip install boto3`
- AWS credentials configured (via environment variables, IAM role, or AWS credentials file)
- A Bedrock model that supports Tool Use (e.g., Claude models)

## Model Compatibility

Not all Bedrock models support Tool Use. Models that **do support** structured outputs include:
- `anthropic.claude-3-5-sonnet-20241022-v2:0`
- `anthropic.claude-3-sonnet-20240229-v1:0`
- `anthropic.claude-3-opus-20240229-v1:0`
- `anthropic.claude-3-haiku-20240307-v1:0`

Check the [Bedrock model documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html) for the latest list of models supporting Tool Use.

## Installation

Install required packages if not already installed:

In [ ]:
%pip install ag2 boto3 pydantic --upgrade

## Setup: Import Libraries and Configure AWS Credentials

In [ ]:
import json
import os

from pydantic import BaseModel, ValidationError

from autogen import ConversableAgent, LLMConfig

# Configure AWS credentials
# Option 1: Environment variables (recommended for local development)
# os.environ["AWS_REGION"] = "us-east-1"
# os.environ["AWS_ACCESS_KEY_ID"] = "your-access-key"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "your-secret-key"

# Option 2: Use IAM role (recommended for EC2, Lambda, ECS)
# No credentials needed - boto3 will use the attached IAM role

# Option 3: AWS credentials file (~/.aws/credentials)
# No code needed - boto3 will automatically use credentials from the file

print("Libraries imported successfully!")

## Part 1: Define Structured Output Models with Pydantic

Pydantic models provide type safety and automatic validation. Let's create a model for math problem solving:

In [ ]:
# Define structured output model for math problem solving
class Step(BaseModel):
    """Represents a single step in solving a math problem."""

    explanation: str  # What operation or reasoning is being performed
    output: str  # The result of this step


class MathReasoning(BaseModel):
    """Complete structured response for a math problem solution."""

    steps: list[Step]  # List of all steps taken
    final_answer: str  # The final answer

    def format(self) -> str:
        """Format the structured output for human-readable display."""
        steps_output = "\n".join(
            f"Step {i + 1}: {step.explanation}\n  Output: {step.output}" for i, step in enumerate(self.steps)
        )
        return f"{steps_output}\n\nFinal Answer: {self.final_answer}"


print("Pydantic models defined:")
print(f"- Step: {Step.model_json_schema()}")
print(f"- MathReasoning: {MathReasoning.model_json_schema()}")

## Part 2: Configure Bedrock with Structured Outputs

Now let's set up the LLM configuration with Bedrock and enable structured outputs:

In [ ]:
# Configure LLM with Bedrock and structured outputs
llm_config = LLMConfig(
    config_list={
        "api_type": "bedrock",
        "model": "anthropic.claude-3-5-sonnet-20241022-v2:0",  # Claude models support tool use
        "aws_region": os.getenv("AWS_REGION", "us-east-1"),
        "aws_access_key": os.getenv("AWS_ACCESS_KEY_ID"),  # Optional if using IAM role
        "aws_secret_key": os.getenv("AWS_SECRET_ACCESS_KEY"),  # Optional if using IAM role
        "response_format": MathReasoning,  # Enable structured outputs with Pydantic model
    },
    cache_seed=42,  # Optional: for reproducible results
)

print("Bedrock LLM configuration created with structured outputs!")

### Key Configuration Parameters

- **`api_type: "bedrock"`**: Tells AG2 to use the Bedrock client
- **`model`**: The Bedrock model ID (must support Tool Use)
- **`aws_region`**: AWS region where Bedrock is available
- **`response_format`**: Your Pydantic model or dict schema - this enables structured outputs

**Note**: When `response_format` is provided, AG2 automatically:
1. Converts your schema into a Bedrock tool definition
2. Forces the model to use this tool via `toolChoice`
3. Extracts and validates the structured response

## Part 3: Create an Agent with Structured Outputs

Create a `ConversableAgent` that will return structured responses:

In [ ]:
# Create agent with structured output capability
math_agent = ConversableAgent(
    name="math_assistant",
    llm_config=llm_config,
    system_message="""You are a helpful math assistant that solves problems step by step.
    Always show your reasoning process clearly with explanations for each step.
    Return your response in the structured format requested.""",
    max_consecutive_auto_reply=1,
)

print(f"Agent '{math_agent.name}' created successfully!")

## Part 4: Example 1 - Simple Equation with Structured Output

Let's solve a simple equation and see the structured response:

In [ ]:
print("=== Example 1: Solve equation with structured output ===")

# Initiate chat with the agent
result1 = math_agent.initiate_chat(
    recipient=math_agent,
    message="Solve the equation: 2x + 5 = -25. Show your work step by step.",
    max_turns=1,
)

# Get the last message from the chat history
last_message = result1.chat_history[-1]["content"]
print("\nRaw response:")
print(last_message)

Now let's parse and validate the structured response:

In [ ]:
# Parse and validate the structured response
try:
    # Parse JSON string
    parsed_response = json.loads(last_message)

    # Validate against Pydantic model
    math_result = MathReasoning.model_validate(parsed_response)

    print("\n✅ Successfully parsed and validated structured response!")
    print("\nFormatted Output:")
    print(math_result.format())

    print("\n\nStructured Data (Pydantic Model):")
    print(f"Number of steps: {len(math_result.steps)}")
    print(f"Final answer: {math_result.final_answer}")

    # Access individual steps
    print("\n\nIndividual Steps:")
    for i, step in enumerate(math_result.steps, 1):
        print(f"  Step {i}: {step.explanation} → {step.output}")

except json.JSONDecodeError as e:
    print(f"❌ Error: Response is not valid JSON: {e}")
    print(f"Raw response: {last_message}")
except ValidationError as e:
    print(f"❌ Error: Response doesn't match schema: {e}")
    print(f"Parsed JSON: {parsed_response}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

## Part 5: Example 2 - Complex Math Problem

Let's try a more complex problem:

In [ ]:
print("=== Example 2: Complex math problem ===")

result2 = math_agent.initiate_chat(
    recipient=math_agent,
    message="Solve: 3(x - 4) + 2x = 5x - 12. Show each step.",
    max_turns=1,
)

try:
    last_message = result2.chat_history[-1]["content"]
    parsed_response = json.loads(last_message)
    math_result = MathReasoning.model_validate(parsed_response)

    print("\n✅ Structured Response:")
    print(math_result.format())

except Exception as e:
    print(f"❌ Error: {e}")

## Part 6: Using Dict Schema Instead of Pydantic Model

You can also use a plain dictionary schema instead of a Pydantic model. This is useful when:
- You don't need Pydantic's validation features
- You want more flexibility in schema definition
- You're working with dynamic schemas

Let's create a different schema for a different use case:

In [ ]:
# Define schema as a dictionary (JSON Schema format)
dict_schema = {
    "type": "object",
    "properties": {
        "problem": {"type": "string", "description": "The math problem being solved"},
        "solution_steps": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {"step": {"type": "string"}, "result": {"type": "string"}},
                "required": ["step", "result"],
            },
        },
        "answer": {"type": "string"},
    },
    "required": ["problem", "solution_steps", "answer"],
}

print("Dict schema defined:")
print(json.dumps(dict_schema, indent=2))

In [ ]:
# Create a new LLM config with dict schema
llm_config_dict = LLMConfig(
    config_list={
        "api_type": "bedrock",
        "model": "anthropic.claude-3-5-sonnet-20241022-v2:0",
        "aws_region": os.getenv("AWS_REGION", "us-east-1"),
        "aws_access_key": os.getenv("AWS_ACCESS_KEY_ID"),
        "aws_secret_key": os.getenv("AWS_SECRET_ACCESS_KEY"),
        "response_format": dict_schema,  # Using dict schema instead of Pydantic model
    },
)

# Create agent with dict schema
math_agent_dict = ConversableAgent(
    name="math_assistant_dict",
    llm_config=llm_config_dict,
    system_message="You are a helpful math assistant.",
    max_consecutive_auto_reply=1,
)

print("Agent created with dict schema!")

In [ ]:
print("=== Example 3: Using dict schema ===")

result3 = math_agent_dict.initiate_chat(
    recipient=math_agent_dict,
    message="Solve: x^2 - 5x + 6 = 0",
    max_turns=1,
)

try:
    last_message = result3.chat_history[-1]["content"]
    parsed_response = json.loads(last_message)

    print("\n✅ Structured Response (from dict schema):")
    print(f"Problem: {parsed_response['problem']}")
    print("\nSolution Steps:")
    for i, step in enumerate(parsed_response["solution_steps"], 1):
        print(f"  {i}. {step['step']} → {step['result']}")
    print(f"\nAnswer: {parsed_response['answer']}")

except (json.JSONDecodeError, KeyError) as e:
    print(f"❌ Error parsing structured output: {e}")
    print(f"Raw response: {last_message}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

## Part 7: Understanding How It Works Under the Hood

When you use `response_format` with Bedrock, AG2:

1. **Converts your schema to a tool**: Your Pydantic model or dict schema becomes a Bedrock tool definition
2. **Forces tool usage**: Sets `toolChoice` to force the model to call the structured output tool
3. **Extracts the data**: Gets the structured data from the tool call's input
4. **Validates**: If using Pydantic, validates the data against your model
5. **Formats**: Returns the JSON string (or formatted string if your model has a `format()` method)

Let's inspect what the tool configuration looks like:

In [ ]:
# Inspect the tool that gets created from your schema
from autogen.oai.bedrock import BedrockClient

# Create a temporary client to see the tool creation
temp_client = BedrockClient(
    aws_region=os.getenv("AWS_REGION", "us-east-1"),
    aws_access_key=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    response_format=MathReasoning,
)

# See how the schema is converted to a tool
tool = temp_client._create_structured_output_tool(MathReasoning)

print("Tool definition created from MathReasoning schema:")
print(json.dumps(tool, indent=2))

Notice that:
- The tool name is `"__structured_output"` (a reserved name)
- The `inputSchema` contains your JSON schema
- The description explains it's for structured output generation

## Part 8: Error Handling

It's important to handle cases where the model might not return valid structured output. Let's see how to handle errors gracefully:

In [ ]:
def safe_structured_chat(agent, message, expected_model):
    """Safely initiate a chat and parse structured output with error handling."""
    try:
        result = agent.initiate_chat(
            recipient=agent,
            message=message,
            max_turns=1,
        )

        last_message = result.chat_history[-1]["content"]

        # Try to parse as JSON
        try:
            parsed = json.loads(last_message)
        except json.JSONDecodeError:
            return {"success": False, "error": "Response is not valid JSON", "raw_response": last_message}

        # Try to validate against model
        try:
            validated = expected_model.model_validate(parsed)
            return {
                "success": True,
                "data": validated,
                "formatted": validated.format() if hasattr(validated, "format") else str(validated),
            }
        except ValidationError as e:
            return {"success": False, "error": f"Validation failed: {e}", "parsed_json": parsed}

    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error: {e}",
        }


# Test the error handling
result = safe_structured_chat(math_agent, "Solve: 10x = 50", MathReasoning)

if result["success"]:
    print("✅ Success!")
    print(result["formatted"])
else:
    print(f"❌ Error: {result['error']}")
    if "raw_response" in result:
        print(f"Raw response: {result['raw_response']}")

## Part 9: Best Practices

### 1. Choose the Right Model
- Use Claude models (they have excellent tool use support)
- Check model compatibility before using structured outputs

### 2. Schema Design
- Keep schemas simple and focused
- Use descriptive field names and descriptions
- Make required fields explicit

### 3. Error Handling
- Always wrap parsing in try/except blocks
- Provide fallback behavior when structured output fails
- Log errors for debugging

### 4. Pydantic vs Dict Schema
- **Use Pydantic** when you need:
  - Type validation
  - Automatic serialization/deserialization
  - IDE autocomplete
  - Custom formatting methods
- **Use Dict Schema** when you need:
  - Dynamic schemas
  - Simpler setup
  - No external dependencies

### 5. Performance Considerations
- Structured outputs add a small overhead (tool call)
- Consider caching for repeated queries
- Use `cache_seed` for reproducible results during development

## Part 10: Advanced Example - Custom Formatting

You can add custom formatting methods to your Pydantic models for better display:

In [ ]:
class DetailedMathReasoning(BaseModel):
    """Enhanced math reasoning with custom formatting."""

    problem: str
    steps: list[Step]
    final_answer: str
    verification: str | None = None

    def format(self) -> str:
        """Custom formatted output with problem statement."""
        output = f"Problem: {self.problem}\n\n"
        output += "Solution Steps:\n"
        for i, step in enumerate(self.steps, 1):
            output += f"  {i}. {step.explanation}\n"
            output += f"     → {step.output}\n"
        output += f"\nFinal Answer: {self.final_answer}"
        if self.verification:
            output += f"\n\nVerification: {self.verification}"
        return output

    def to_markdown(self) -> str:
        """Export as Markdown format."""
        md = f"## Problem\n\n{self.problem}\n\n"
        md += "## Solution\n\n"
        for i, step in enumerate(self.steps, 1):
            md += f"### Step {i}\n\n"
            md += f"**Explanation**: {step.explanation}\n\n"
            md += f"**Result**: `{step.output}`\n\n"
        md += f"## Final Answer\n\n`{self.final_answer}`"
        return md


# Create agent with enhanced model
enhanced_llm_config = LLMConfig(
    config_list={
        "api_type": "bedrock",
        "model": "anthropic.claude-3-5-sonnet-20241022-v2:0",
        "aws_region": os.getenv("AWS_REGION", "us-east-1"),
        "aws_access_key": os.getenv("AWS_ACCESS_KEY_ID"),
        "aws_secret_key": os.getenv("AWS_SECRET_ACCESS_KEY"),
        "response_format": DetailedMathReasoning,
    },
)

enhanced_agent = ConversableAgent(
    name="enhanced_math_assistant",
    llm_config=enhanced_llm_config,
    system_message="You are a detailed math assistant. Always verify your answers.",
    max_consecutive_auto_reply=1,
)

print("Enhanced agent created with custom formatting!")

In [ ]:
# Test the enhanced agent
result = enhanced_agent.initiate_chat(
    recipient=enhanced_agent,
    message="Solve: 4x - 8 = 12. Show your work and verify the answer.",
    max_turns=1,
)

try:
    last_message = result.chat_history[-1]["content"]
    parsed = json.loads(last_message)
    math_result = DetailedMathReasoning.model_validate(parsed)

    print("\n=== Formatted Output ===")
    print(math_result.format())

    print("\n\n=== Markdown Export ===")
    print(math_result.to_markdown())

except Exception as e:
    print(f"Error: {e}")

## Summary

In this notebook, we've learned:

1. ✅ How to define structured output schemas using Pydantic models
2. ✅ How to configure Bedrock with `response_format` for structured outputs
3. ✅ How to create agents that return structured, parseable responses
4. ✅ How to parse and validate structured responses
5. ✅ How to use dict schemas as an alternative to Pydantic
6. ✅ How structured outputs work under the hood with Bedrock Tool Use
7. ✅ Best practices for error handling and schema design
8. ✅ Advanced techniques like custom formatting methods

## Next Steps

- Try creating your own structured output schemas for different use cases
- Experiment with different Bedrock models that support Tool Use
- Combine structured outputs with other AG2 features like multi-agent conversations
- Explore using structured outputs for data extraction and analysis tasks

## References

- [AG2 Documentation](https://docs.ag2.ai)
- [Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)
- [AWS Blog: Structured Data with Bedrock](https://aws.amazon.com/blogs/machine-learning/structured-data-response-with-amazon-bedrock-prompt-engineering-and-tool-use/)
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [Bedrock Model IDs](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html)